In [1]:
import pandas as pd
from datetime import datetime, timedelta
import os

# this code creates a trip to stops mapping table from all gtfs files

In [2]:
# retrieve stop_times.txt files for each gtfs generation dates
extract_path = "./files/extracted/"
stop_times_path = "/stop_times.txt"
stops_path = "/stops.txt"
gtfs_generation_dates = [item for item in os.listdir(extract_path) if os.path.isdir(os.path.join(extract_path, item))]

mappings_path = "./mappings/"
services_to_trips_df = pd.read_csv(mappings_path + 'services_to_trips.csv')
stops_to_stations_df = pd.read_csv(mappings_path + 'stops_to_stations.csv')

In [3]:
def process_stop_times_file():

    agg_stop_times = pd.DataFrame()
    for gtfs_date in gtfs_generation_dates:

        # Read the stop_times.txt file into a DataFrame
        stop_times = pd.read_csv(extract_path + gtfs_date + stop_times_path)
        agg_stop_times = pd.concat([agg_stop_times, stop_times])

    agg_stop_times = agg_stop_times[['trip_id','departure_time','stop_id','stop_sequence']] \
        .drop_duplicates() # \
        # .sort_values(by=['trip_id','departure_time','stop_id','stop_sequence'])
    return(agg_stop_times)

# reduces total number from 31593819 to 10850498 

In [4]:
def process_stops_file():

    agg_stops = pd.DataFrame()
    for gtfs_date in gtfs_generation_dates:

        # Read the stop_times.txt file into a DataFrame
        stops = pd.read_csv(extract_path + gtfs_date + stops_path)
        agg_stops = pd.concat([agg_stops, stops])

    agg_stops = agg_stops[['stop_id','parent_station']] \
        .drop_duplicates()
    return(agg_stops)

In [5]:
stop_times_df = process_stop_times_file()
stop_times_df

,trip_id,departure_time,stop_id,stop_sequence
0,A20130803WKD_000800_1..S03R,00:08:00,101S,1
1,A20130803WKD_000800_1..S03R,00:09:00,103S,2
2,A20130803WKD_000800_1..S03R,00:10:24,104S,3
3,A20130803WKD_000800_1..S03R,00:11:48,106S,4
4,A20130803WKD_000800_1..S03R,00:13:12,107S,5
...,...,...,...,...
535382,BFA22GEN-R092-Weekday-00_081800_R..N96R,14:46:30,G12N,34
535383,BFA22GEN-R092-Weekday-00_081800_R..N96R,14:48:30,G11N,35
535384,BFA22GEN-R092-Weekday-00_081800_R..N96R,14:50:00,G10N,36
535385,BFA22GEN-R092-Weekday-00_081800_R..N96R,14:52:00,G09N,37


In [6]:
stops_df = process_stops_file()
stops_df

,stop_id,parent_station
0,101,NaN
1,101N,101
2,101S,101
3,103,NaN
4,103N,103
...,...,...
1435,S11N,S11
1436,S11S,S11
1086,H19,NaN
1087,H19N,H19


In [7]:
# Merge the stops dataframe into the stop_times dataframe on the stop_id column
parent_stop_times_df = pd.merge(stop_times_df, stops_df[['stop_id', 'parent_station']], on='stop_id')

# Replace the stop_id column with the parent_station column
parent_stop_times_df['stop_id'] = parent_stop_times_df['parent_station']

# Drop the parent_station column, which is no longer needed
parent_stop_times_df = parent_stop_times_df.drop('parent_station', axis=1)

parent_stop_times_df

,trip_id,departure_time,stop_id,stop_sequence
0,A20130803WKD_000800_1..S03R,00:08:00,101,1
1,A20130803WKD_002700_1..S03R,00:27:00,101,1
2,A20130803WKD_004700_1..S03R,00:47:00,101,1
3,A20130803WKD_006700_1..S03R,01:07:00,101,1
4,A20130803WKD_008700_1..S03R,01:27:00,101,1
...,...,...,...,...
10850493,SIR-FA2017-SI017-Sunday-00_135100_SI..N03R,22:34:00,S11,2
10850494,SIR-FA2017-SI017-Sunday-00_147100_SI..N03R,24:34:00,S11,2
10850495,SIR-FA2017-SI017-Sunday-00_015100_SI..N03R,02:34:00,S11,2
10850496,SIR-FA2017-SI017-Sunday-00_021100_SI..N03R,03:34:00,S11,2


In [8]:
# Merge the stop_times_df dataframe into the parent_stop_times_df dataframe on the stop_id column
station_times_df = pd.merge(parent_stop_times_df, stops_to_stations_df[['stop_id', 'station_id']], on='stop_id')

# Drop the stop_id column, which is no longer needed
station_times_df = station_times_df.drop('stop_id', axis=1) \
    .sort_values(['trip_id','stop_sequence']) \
    .reset_index(drop=True)

station_times_df

,trip_id,departure_time,stop_sequence,station_id
0,A20130803SAT_000300_2..S08R,00:03:00,1,201
1,A20130803SAT_000300_2..S08R,00:04:30,2,204
2,A20130803SAT_000300_2..S08R,00:05:48,3,205
3,A20130803SAT_000300_2..S08R,00:07:06,4,206
4,A20130803SAT_000300_2..S08R,00:08:18,5,207
...,...,...,...,...
10572001,montague_wkd_153700_N..N20R,26:57:00,41,R06
10572002,montague_wkd_153700_N..N20R,26:59:00,42,R05
10572003,montague_wkd_153700_N..N20R,27:00:30,43,R04
10572004,montague_wkd_153700_N..N20R,27:02:00,44,R03


In [9]:
# convert time to timedelta object
station_times_df['departure_time'] = pd.to_timedelta(station_times_df['departure_time'])

# add 1 day to the time with hour component > 23
station_times_df.loc[station_times_df['departure_time'] > pd.Timedelta(hours=23), 'departure_time'] += pd.Timedelta(days=1)

# convert timedelta to total number of seconds
station_times_df['departure_time_seconds'] = station_times_df['departure_time'].dt.total_seconds()

station_times_df

,trip_id,departure_time,stop_sequence,station_id,departure_time_seconds
0,A20130803SAT_000300_2..S08R,0 days 00:03:00,1,201,180.0
1,A20130803SAT_000300_2..S08R,0 days 00:04:30,2,204,270.0
2,A20130803SAT_000300_2..S08R,0 days 00:05:48,3,205,348.0
3,A20130803SAT_000300_2..S08R,0 days 00:07:06,4,206,426.0
4,A20130803SAT_000300_2..S08R,0 days 00:08:18,5,207,498.0
...,...,...,...,...,...
10572001,montague_wkd_153700_N..N20R,2 days 02:57:00,41,R06,183420.0
10572002,montague_wkd_153700_N..N20R,2 days 02:59:00,42,R05,183540.0
10572003,montague_wkd_153700_N..N20R,2 days 03:00:30,43,R04,183630.0
10572004,montague_wkd_153700_N..N20R,2 days 03:02:00,44,R03,183720.0


In [10]:
station_graph_df = station_times_df

# Use shift 
station_graph_df['station_from'] = station_graph_df['station_id']
station_graph_df['station_to'] = station_graph_df['station_id'].shift(-1)

station_graph_df['transit_duration_seconds'] = station_graph_df['departure_time_seconds'].shift(-1) - station_graph_df['departure_time_seconds']

station_graph_df.dropna(subset=['station_to'], inplace=True)
station_graph_df.drop(['stop_sequence', 'station_id'], axis=1, inplace=True)
station_graph_df = station_graph_df[['trip_id', 'station_from', 'station_to', 'transit_duration_seconds']]
station_graph_df

,trip_id,station_from,station_to,transit_duration_seconds
0,A20130803SAT_000300_2..S08R,201,204,90.0
1,A20130803SAT_000300_2..S08R,204,205,78.0
2,A20130803SAT_000300_2..S08R,205,206,78.0
3,A20130803SAT_000300_2..S08R,206,207,72.0
4,A20130803SAT_000300_2..S08R,207,208,84.0
...,...,...,...,...
10572000,montague_wkd_153700_N..N20R,R08,R06,60.0
10572001,montague_wkd_153700_N..N20R,R06,R05,120.0
10572002,montague_wkd_153700_N..N20R,R05,R04,90.0
10572003,montague_wkd_153700_N..N20R,R04,R03,90.0


In [11]:
# Generate a mappings folder if it doesn't exist
if not os.path.exists('mappings'):
    os.makedirs('mappings')

# Generate a mapping file for future reference
station_graph_df.to_csv("mappings/trips_to_station_nodes_edges.csv", index=False)